https://github.com/Student204161/computational-social-science-assignments
Part 1

In [ ]:
from bs4 import BeautifulSoup ##A package to work with HTML data
import requests #A package to make HTTP requests

# Code to get the participants from the oral presentation
LINK = "https://2019.ic2s2.org/oral-presentations/"
r = requests.get(LINK)
soup = BeautifulSoup(r.content)
# find dev elements based on a specific class
div = soup.find("div",{"class":"col-md-8 page-content-wrap"})
div_p = div.find_all("p")[3:]
for d in div_p:
    d.strong.extract()

# Filter and clean diferent elements to get the participants
data = []
for d in div_p:
    output = d.get_text(strip=True,separator="\n").splitlines()[1:]
    for out in output:
        if out[0].isdigit():
            removed_date = list(filter(None, out.split("–")[2:]))[0]
            removed_topic = removed_date.split(".")[0]
            researchers = removed_topic.split(",")
            for r in researchers:
                data.append(r)
        elif out.split(": ")[0] == "Chair":
            data.append(out.split(": ")[1])

data_unique = list(set(data))

In [ ]:
# Code to get the participants from the posters
LINK = "https://2019.ic2s2.org/posters/"
r = requests.get(LINK)
soup = BeautifulSoup(r.content)
div = soup.find("div",{"class":"col-md-8 page-content-wrap"})
# Filter and clean diferent elements to get the participants
for child in div.find("ul").children:
    txt = child.get_text(strip=True,separator="\n").split("<span>")[0].split("\n")[0]
    txt = txt.replace(" and ", ", ")
    txt_split = txt.split(", ")
    for t in txt_split:
        data.append(t)
    print("-----------------------------------")
    print("child:")
    print(child)
data_unique = list(set(data))
while "" in data_unique:
    data_unique.remove("")

798 unique participants obtained in 2019